In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep

In [20]:
url = 'https://www.thegioididong.com/sac-dtdd'

driver = webdriver.Chrome()
driver.get(url)

### Hiển thị tất cả sản phẩm

In [21]:
show_more_elm = driver.find_element(By.XPATH, '//*[@id="categoryPage"]/div[1]/div[2]/a')
show_more_elm

<selenium.webdriver.remote.webelement.WebElement (session="47a68cb0f6f433d756977a588753f3e9", element="f.CC57D641897AA37F4A02343D26CA2EFD.d.1C58F64D962ECC9B629792839959BE8F.e.105")>

In [22]:
while True:
    try:
        show_more_elm.click()
        sleep(2)
    except:
        print('We are at the end of the page!')
        break

We are at the end of the page!


### Lấy danh sách sản phẩm

In [23]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

72

#### Tạo Func lấy thông 1 tin sản phẩm

In [24]:
def get_product_info(product_elm):
    phone_name = product_elm.find_element(By.XPATH, './/h3').text

    try:
        phone_price = product_elm.find_element(By.XPATH, './/strong[@class="price"]').text
    except:
        phone_price = "N/A"

    result = {
        'name': phone_name,
        'price': phone_price
    }
    return result

In [37]:
# Hàm lấy review của tất cả điện thoại
all_reviews = []
tmp = 0
def getReviewAll(product_list):
    for i in range(65, len(product_list)):
        product_info = get_product_info(product_list[i])
        product_list[i].click()
        sleep(2)

        try:
            # Nếu mà có readmore thì mình bấm luôn
            try:
                read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[9]/div[2]/div/div/div[6]/div/a')
            except:
                read_more_elm = driver.find_element(By.XPATH, '/html/body/section/div[2]/div[1]/div[11]/div[2]/div/div/div[6]/div/a')
            if read_more_elm:
                read_more_elm.click()
                sleep(2)

                while True:
                    reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                    for review in reviews:
                        all_reviews.append({
                            'product_name': product_info['name'],
                            'product_price': product_info['price'],
                            'review': review.text
                        })

                    try:
                        page_elm = driver.find_element(By.CLASS_NAME, 'pagcomment')
                        list_page = page_elm.find_elements(By.XPATH, './/a')

                        next_page = None # Init var
                        for page in list_page:
                            if page.text.strip() == "»" or page.text.strip() == "›":
                                next_page = page
                                break

                        if next_page: # if next_page exist
                            next_page.click() # forward
                            sleep(3)
                        else:
                            last_page = list_page[-1]
                            last_page.click()
                            sleep(3)
                            # Lấy xong hết review của trang cuối cùng của sản phẩm
                            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
                            for review in reviews:
                                all_reviews.append({
                                    'product_name': product_info['name'],
                                    'product_price': product_info['price'],
                                    'review': review.text
                                })
                            driver.back()
                            driver.back()
                            sleep(3)
                            break
                    except:
                        print("Không còn trang tiếp theo!")
                        driver.back()
                        driver.back()
                        sleep(2)
                        break
        except:

            # Nếu có review nhưng chỉ có ===>> 1 TRANG
            reviews = driver.find_elements(By.CLASS_NAME, 'cmt-txt')
            for review in reviews:
                all_reviews.append({
                    'product_name': product_info['name'],
                    'product_price': product_info['price'],
                    'review': review.text
                })
            
            # Lấy xong thì back về
            driver.back()
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')

In [38]:
getReviewAll(product_list)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".listproduct"}
  (Session info: chrome=134.0.6998.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF79DEF4C25+3179557]
	(No symbol) [0x00007FF79DB588A0]
	(No symbol) [0x00007FF79D9E91CA]
	(No symbol) [0x00007FF79DA3FA67]
	(No symbol) [0x00007FF79DA3FC9C]
	(No symbol) [0x00007FF79DA93627]
	(No symbol) [0x00007FF79DA67C6F]
	(No symbol) [0x00007FF79DA902F3]
	(No symbol) [0x00007FF79DA67A03]
	(No symbol) [0x00007FF79DA306D0]
	(No symbol) [0x00007FF79DA31983]
	GetHandleVerifier [0x00007FF79DF567CD+3579853]
	GetHandleVerifier [0x00007FF79DF6D1D2+3672530]
	GetHandleVerifier [0x00007FF79DF62153+3627347]
	GetHandleVerifier [0x00007FF79DCC092A+868650]
	(No symbol) [0x00007FF79DB62FFF]
	(No symbol) [0x00007FF79DB5F4A4]
	(No symbol) [0x00007FF79DB

In [39]:
df = pd.DataFrame(all_reviews)
df.to_csv('D:\CrawlData\Sentiment_Review\data\cmt_backup_charger_continute_1.csv', index=False)